Data loaders

In [34]:
import numpy as np
import struct

def load_mnist_images(filename):
    with open(filename, "rb") as f:
        magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, -1)
    return images

def load_mnist_labels(filename):
    with open(filename, "rb") as f:
        magic, num_labels = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

In [35]:
train_images = load_mnist_images("train-images.idx3-ubyte")
train_labels = load_mnist_labels("train-labels.idx1-ubyte")

test_images = load_mnist_images("t10k-images.idx3-ubyte")
test_labels = load_mnist_labels("t10k-labels.idx1-ubyte")

MNIST Classifier Interface

In [36]:
from abc import ABC, abstractmethod

class MnistClassifierInterface(ABC):  
    @abstractmethod
    def train(self, X, y):
        # scratch method for model training
        pass  

    @abstractmethod
    def predict(self, X):
        # scratch method for prediction
        pass  

Random Forest classifier

In [37]:
class RandomForestMnistClassifier(MnistClassifierInterface):

    def __init__(self):
        self.model = None

    def train(self, X, y):
        from sklearn.ensemble import RandomForestClassifier
        self.model = RandomForestClassifier(n_estimators=150, random_state=42)
        self.model.fit(X, y)

    def predict(self, X):
        """метод для реалізації передбачення"""
        if self.model is None:
            return None
        else:
            return self.model.predict(X)

Feed-Forward NN classifier

In [38]:
class FeedForwardNeuralNetworkClassifier(MnistClassifierInterface):

    def __init__(self):
        self.model = None

    def train(self, X, y):
        from sklearn.neural_network import MLPClassifier
        self.model = MLPClassifier(hidden_layer_sizes = (100,), activation = 'relu', solver= 'adam', max_iter = 200)
        self.model.fit(X, y)

    def predict(self, X):
        """метод для реалізації передбачення"""
        if self.model is None:
            return None
        else:
            return self.model.predict(X)

Convolutional NN classifier

In [39]:
class ConvolutionalNeuralNetworkClassifier(MnistClassifierInterface):

    def __init__(self):
        self.model = None

    def train(self, X, y):
        from keras.models import Sequential
        from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
        self.model = Sequential()
        # Convolutional and pooling layers addition
        input_layer = Input(shape=(28, 28, 1))
        conv_layer = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')
        pool_layer = MaxPooling2D(pool_size=(2, 2))
        # Flat layer addition
        flat_layer = Flatten()
        # Dense layer addition
        dense_layer_one = Dense(64, activation='relu')
        dense_layer_two = Dense(10, activation='softmax')
        self.model.add(input_layer)
        self.model.add(conv_layer)
        self.model.add(pool_layer)
        self.model.add(flat_layer)
        self.model.add(dense_layer_one)
        self.model.add(dense_layer_two)
        # Model compilation
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
        self.model.fit(X, y)

    def predict(self, X):
        """метод для реалізації передбачення"""
        if self.model is None:
            return None
        else:
            probabilities = self.model.predict(X)
            return np.argmax(probabilities, axis=1)

Reshape images for CNN classification

In [40]:
train_images_cnn = train_images.reshape(-1, 28, 28, 1)
test_images_cnn = test_images.reshape(-1, 28, 28, 1)

MNIST classifier

In [41]:
from sklearn.metrics import accuracy_score

class MnistClassifier:

    def __init__(self, algorithm):
        if algorithm == 'rf':
            self.model = RandomForestMnistClassifier()
        elif algorithm == 'nn':
            self.model = FeedForwardNeuralNetworkClassifier()
        elif algorithm == 'cnn':
            self.model = ConvolutionalNeuralNetworkClassifier()
        else:
            raise ValueError("Algorithm is not found, choose from rf, nn or cnn")

    def train(self, X, y):
        self.model.train(X, y)

    def predict(self, X):
        return self.model.predict(X)

In [42]:
from MnistClassifier import MnistClassifier


def run():
    algorithm = input("Enter wanted algorithm (available options - rf, nn, cnn) or 'exit' to quit: ").lower()

    if algorithm.lower() == 'exit':
        print("Exiting the program.")
        return  # Ends program if user wants us to

    try:
        print("Processing...")
        model = MnistClassifier(algorithm)
        model.train()
        print("Model accuracy: ", model.accuracy())
    except ValueError as e:
        print(e)  # Outputs "Algorithm is not found", if algorithm name is wrong
    except Exception as e:
        print("An error occurred: ", e)  # For other errors

    run()


if __name__ == '__main__':
    run()

Enter wanted algorithm (available options - rf, nn, cnn) or 'exit' to quit:  rf


Processing...
Model accuracy:  0.9707


Enter wanted algorithm (available options - rf, nn, cnn) or 'exit' to quit:  nn


Processing...
Model accuracy:  0.9694


Enter wanted algorithm (available options - rf, nn, cnn) or 'exit' to quit:  cnn


Processing...
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - loss: 1.3474
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Model accuracy:  0.9724


Enter wanted algorithm (available options - rf, nn, cnn) or 'exit' to quit:  exit


Exiting the program.


Edge Cases

1. Invalid Algorithm Input:

If the user inputs an invalid algorithm name (not rf, nn, or cnn), a ValueError will be raised, indicating that the algorithm is not found.

2. Empty Dataset:

If the dataset is empty or improperly formatted, the model will raise errors during training or prediction. Ensure that the MNIST files are correctly extracted and in the right format.

3. Model Not Trained:

If you attempt to make a prediction before training the model, the predict method will return None.

4. Incompatible Input Shape:

The CNN expects input images to be reshaped to (28, 28, 1). Providing images in a different shape will result in errors during model training.